In [2]:
import IPython
import numpy as np
import tvm
from tvm.ir.module import IRModule
from tvm.script import tir as T

# 示例：逐位相加

In [5]:
# init data
a = np.arange(16).reshape(4, 4)
b = np.arange(16, 0, -1).reshape(4, 4)
c_np = a + b

In [6]:
# TensorIR version
@tvm.script.ir_module
class MyAdd:
  @T.prim_func
  def add(A: T.Buffer[(4, 4), "int64"],
          B: T.Buffer[(4, 4), "int64"],
          C: T.Buffer[(4, 4), "int64"]):
    T.func_attr({"global_symbol": "add"})
    for i, j in T.grid(4, 4):
      with T.block("C"):
        vi = T.axis.spatial(4, i)
        vj = T.axis.spatial(4, j)
        C[vi, vj] = A[vi, vj] + B[vi, vj]

rt_lib = tvm.build(MyAdd, target="llvm")
a_tvm = tvm.nd.array(a)
b_tvm = tvm.nd.array(b)
c_tvm = tvm.nd.array(np.empty((4, 4), dtype=np.int64))
rt_lib["add"](a_tvm, b_tvm, c_tvm)
np.testing.assert_allclose(c_tvm.numpy(), c_np, rtol=1e-5)

In [7]:
a = np.arange(16).reshape(4, 4)
b = np.arange(4, 0, -1).reshape(4)
c_np = a + b

# 练习 1：广播加法

In [10]:
@tvm.script.ir_module
class MyAdd:
  @T.prim_func
  def add(A: T.Buffer[(4, 4), "int64"],
          B: T.Buffer[(4), "int64"],
          C: T.Buffer[(4, 4), "int64"]):
    T.func_attr({"global_symbol": "add", "tir.noalias": True})
    # TODO
    for i, j in T.grid(4,4):
      with T.block("C"):
        vi = T.axis.spatial(4, i)
        vj = T.axis.spatial(4, j)
        C[vi, vj] = A[vi,vj] + B[vj]
        

rt_lib = tvm.build(MyAdd, target="llvm")
a_tvm = tvm.nd.array(a)
b_tvm = tvm.nd.array(b)
c_tvm = tvm.nd.array(np.empty((4, 4), dtype=np.int64))
rt_lib["add"](a_tvm, b_tvm, c_tvm)
np.testing.assert_allclose(c_tvm.numpy(), c_np, rtol=1e-5)

# 练习 2：二维卷积

In [9]:
N, CI, H, W, CO, K = 1, 1, 8, 8, 2, 3
OUT_H, OUT_W = H - K + 1, W - K + 1
data = np.arange(N*CI*H*W).reshape(N, CI, H, W)
weight = np.arange(CO*CI*K*K).reshape(CO, CI, K, K)

In [10]:
import torch

data_torch = torch.Tensor(data)
weight_torch = torch.Tensor(weight)
conv_torch = torch.nn.functional.conv2d(data_torch, weight_torch)
conv_torch = conv_torch.numpy().astype(np.int64)
conv_torch

array([[[[ 474,  510,  546,  582,  618,  654],
         [ 762,  798,  834,  870,  906,  942],
         [1050, 1086, 1122, 1158, 1194, 1230],
         [1338, 1374, 1410, 1446, 1482, 1518],
         [1626, 1662, 1698, 1734, 1770, 1806],
         [1914, 1950, 1986, 2022, 2058, 2094]],

        [[1203, 1320, 1437, 1554, 1671, 1788],
         [2139, 2256, 2373, 2490, 2607, 2724],
         [3075, 3192, 3309, 3426, 3543, 3660],
         [4011, 4128, 4245, 4362, 4479, 4596],
         [4947, 5064, 5181, 5298, 5415, 5532],
         [5883, 6000, 6117, 6234, 6351, 6468]]]])

In [16]:
@tvm.script.ir_module
class MyConv:
  @T.prim_func
  def conv(data:T.Buffer[(N,CI,8,8), "int64"],
            weight:T.Buffer[(CO,CI,K,K), "int64"],
            output:T.Buffer[(N,CO,OUT_H,OUT_W), "int64"]):
    T.func_attr({"global_symbol": "conv", "tir.noalias": True})
    # TODO
    for b,k,i,j,di,dj,q in T.grid(N,CO,OUT_H,OUT_W,K,K,CI):
      with T.block("C"):
        vb = T.axis.spatial(N, b)
        vk = T.axis.spatial(CO, k)
        vi = T.axis.spatial(OUT_H, i)
        vj = T.axis.spatial(OUT_W, j)
        vdi = T.axis.reduce(K, di)
        vdj = T.axis.reduce(K, dj)
        vq = T.axis.reduce(CI, q)
        with T.init():
          output[vb,vk,vi,vj] = T.int64(0)
        output[vb,vk,vi,vj] = output[vb,vk,vi,vj] + data[vb, vq,vi+vdi,vj+vdj] * weight[vk,vq,vdi,vdj]
        
        

rt_lib = tvm.build(MyConv, target="llvm")
data_tvm = tvm.nd.array(data)
weight_tvm = tvm.nd.array(weight)
conv_tvm = tvm.nd.array(np.empty((N, CO, OUT_H, OUT_W), dtype=np.int64))
rt_lib["conv"](data_tvm, weight_tvm, conv_tvm)
# print(conv_tvm.numpy())
np.testing.assert_allclose(conv_tvm.numpy(), conv_torch, rtol=1e-5)

# 并行化、向量化与循环展开

In [17]:
@tvm.script.ir_module
class MyAdd:
  @T.prim_func
  def add(A: T.Buffer[(4, 4), "int64"],
          B: T.Buffer[(4, 4), "int64"],
          C: T.Buffer[(4, 4), "int64"]):
    T.func_attr({"global_symbol": "add"})
    for i, j in T.grid(4, 4):
      with T.block("C"):
        vi = T.axis.spatial(4, i)
        vj = T.axis.spatial(4, j)
        C[vi, vj] = A[vi, vj] + B[vi, vj]

sch = tvm.tir.Schedule(MyAdd)
block = sch.get_block("C", func_name="add")
i, j = sch.get_loops(block)
i0, i1 = sch.split(i, factors=[2, 2])
sch.parallel(i0)
sch.unroll(i1)
sch.vectorize(j)
IPython.display.Code(sch.mod.script(), language="python")

@tvm.script.ir_module
class Module:
    @T.prim_func
    def add(A: T.Buffer[(4, 4), "int64"], B: T.Buffer[(4, 4), "int64"], C: T.Buffer[(4, 4), "int64"]) -> None:
        # function attr dict
        T.func_attr({"global_symbol": "add"})
        # body
        # with T.block("root")
        for i_0 in T.parallel(2):
            for i_1 in T.unroll(2):
                for j in T.vectorized(4):
                    with T.block("C"):
                        vi = T.axis.spatial(4, i_0 * 2 + i_1)
                        vj = T.axis.spatial(4, j)
                        T.reads(A[vi, vj], B[vi, vj])
                        T.writes(C[vi, vj])
                        C[vi, vj] = A[vi, vj] + B[vi, vj]

# 变换批量矩阵乘法程序

In [17]:
@tvm.script.ir_module
class MyBmmRelu:
  @T.prim_func
  def bmm_relu(A: T.Buffer[(16, 128, 128), "float32"],
          B: T.Buffer[(16, 128, 128), "float32"],
          C: T.Buffer[(16, 128, 128), "float32"]):
    T.func_attr({"global_symbol": "bmm_relu", "tir.noalias": True})
    # TODO
    Y = T.alloc_buffer((16, 128, 128), dtype="float32")
    for b, i, j, k in T.grid(16,128,128,128):
        with T.block("Y"):
            vb = T.axis.spatial(16, b)
            vi = T.axis.spatial(128, i)
            vj = T.axis.spatial(128, j)
            vk = T.axis.reduce(128, k)
            with T.init():
                Y[vb, vi, vj] = T.float32(0)
            Y[vb, vi, vj] = Y[vb, vi, vj] + A[vb, vi, vk] * B[vb, vk, vj]

    for b, i, j in T.grid(16,128,128):
        with T.block("C"):
            vb = T.axis.spatial(16, b)
            vi = T.axis.spatial(128, i)
            vj = T.axis.spatial(128, j)
            C[vb, vi, vj] = T.max(Y[vb, vi, vj], T.float32(0))
sch = tvm.tir.Schedule(MyBmmRelu)
IPython.display.Code(sch.mod.script(), language="python")
# Also please validate your result

@tvm.script.ir_module
class Module:
    @T.prim_func
    def bmm_relu(A: T.Buffer[(16, 128, 128), "float32"], B: T.Buffer[(16, 128, 128), "float32"], C: T.Buffer[(16, 128, 128), "float32"]) -> None:
        # function attr dict
        T.func_attr({"global_symbol": "bmm_relu", "tir.noalias": True})
        # body
        # with T.block("root")
        Y = T.alloc_buffer([16, 128, 128], dtype="float32")
        for b, i, j, k in T.grid(16, 128, 128, 128):
            with T.block("Y"):
                vb, vi, vj, vk = T.axis.remap("SSSR", [b, i, j, k])
                T.reads(A[vb, vi, vk], B[vb, vk, vj])
                T.writes(Y[vb, vi, vj])
                with T.init():
                    Y[vb, vi, vj] = T.float32(0)
                Y[vb, vi, vj] = Y[vb, vi, vj] + A[vb, vi, vk] * B[vb, vk, vj]
        for b, i, j in T.grid(16, 128, 128):
            with T.block("C"):
                vb, vi, vj = T.axis.remap("SSS", [b, i, j])
                T.reads(Y[vb, vi, vj])
                T.writes(C[vb, vi, vj])
                C[vb, vi, vj] = T.max(Y[vb, vi, vj], T.float32(0))

In [5]:
@tvm.script.ir_module
class TargetModule:
    @T.prim_func
    def bmm_relu(A: T.Buffer[(16, 128, 128), "float32"], B: T.Buffer[(16, 128, 128), "float32"], C: T.Buffer[(16, 128, 128), "float32"]) -> None:
        T.func_attr({"global_symbol": "bmm_relu", "tir.noalias": True})
        Y = T.alloc_buffer([16, 128, 128], dtype="float32")
        for i0 in T.parallel(16):
            for i1, i2_0 in T.grid(128, 16):
                for ax0_init in T.vectorized(8):
                    with T.block("Y_init"):
                        n, i = T.axis.remap("SS", [i0, i1])
                        j = T.axis.spatial(128, i2_0 * 8 + ax0_init)
                        Y[n, i, j] = T.float32(0)
                for ax1_0 in T.serial(32):
                    for ax1_1 in T.unroll(4):
                        for ax0 in T.serial(8):
                            with T.block("Y_update"):
                                n, i = T.axis.remap("SS", [i0, i1])
                                j = T.axis.spatial(128, i2_0 * 8 + ax0)
                                k = T.axis.reduce(128, ax1_0 * 4 + ax1_1)
                                Y[n, i, j] = Y[n, i, j] + A[n, i, k] * B[n, k, j]
                for i2_1 in T.vectorized(8):
                    with T.block("C"):
                        n, i = T.axis.remap("SS", [i0, i1])
                        j = T.axis.spatial(128, i2_0 * 8 + i2_1)
                        C[n, i, j] = T.max(Y[n, i, j], T.float32(0))

In [40]:
sch = tvm.tir.Schedule(MyBmmRelu)
# TODO: transformations
# Hints: you can use
# `IPython.display.Code(sch.mod.script(), language="python")`
# or `print(sch.mod.script())`
# to show the current program at any time during the transformation.

# Step 1. Get blocks
Y = sch.get_block("Y", func_name="bmm_relu")

# Step 2. Get loops
b, i, j, k = sch.get_loops(Y)
sch.parallel(b)

# Step 3. Organize the loops
k0, k1 = sch.split(k, [None, 4])
j0, j1 = sch.split(j, [None, 8])
sch.reorder(j0, k0, k1, j1)

block_C = sch.get_block("C", "bmm_relu")
sch.reverse_compute_at(block_C, j0)

# Step 4. decompose reduction
block_Y = sch.get_block("Y", "bmm_relu")
sch.decompose_reduction(block_Y, k0)


# Step 5. vectorize / parallel / unroll
Y_init = sch.get_block("Y_init", func_name="bmm_relu")
_, _, _, j_1_init = sch.get_loops(Y_init)

C = sch.get_block("C", "bmm_relu")
_, _, _, ax0 = sch.get_loops(C)
# 
sch.vectorize(j_1_init)
sch.vectorize(ax0)
sch.unroll(k1)

IPython.display.Code(sch.mod.script(), language="python")

@tvm.script.ir_module
class Module:
    @T.prim_func
    def bmm_relu(A: T.Buffer[(16, 128, 128), "float32"], B: T.Buffer[(16, 128, 128), "float32"], C: T.Buffer[(16, 128, 128), "float32"]) -> None:
        # function attr dict
        T.func_attr({"global_symbol": "bmm_relu", "tir.noalias": True})
        # body
        # with T.block("root")
        Y = T.alloc_buffer([16, 128, 128], dtype="float32")
        for b in T.parallel(16):
            for i, j_0 in T.grid(128, 16):
                for j_1_init in T.vectorized(8):
                    with T.block("Y_init"):
                        vb, vi = T.axis.remap("SS", [b, i])
                        vj = T.axis.spatial(128, j_0 * 8 + j_1_init)
                        T.reads()
                        T.writes(Y[vb, vi, vj])
                        Y[vb, vi, vj] = T.float32(0)
                for k_0 in T.serial(32):
                    for k_1 in T.unroll(4):
                        for j_1 in T.serial(8):
                            with T.block("Y_update"):
                                vb, vi = T.axis.remap("SS", [b, i])
                                vj = T.axis.spatial(128, j_0 * 8 + j_1)
                                vk = T.axis.reduce(128, k_0 * 4 + k_1)
                                T.reads(Y[vb, vi, vj], A[vb, vi, vk], B[vb, vk, vj])
                                T.writes(Y[vb, vi, vj])
                                Y[vb, vi, vj] = Y[vb, vi, vj] + A[vb, vi, vk] * B[vb, vk, vj]
                for ax0 in T.vectorized(8):
                    with T.block("C"):
                        vb, vi = T.axis.remap("SS", [b, i])
                        vj = T.axis.spatial(128, j_0 * 8 + ax0)
                        T.reads(Y[vb, vi, vj])
                        T.writes(C[vb, vi, vj])
                        C[vb, vi, vj] = T.max(Y[vb, vi, vj], T.float32(0))

In [41]:
tvm.ir.assert_structural_equal(sch.mod, TargetModule)
print("Pass")

Pass


In [42]:
def lnumpy_mm_relu_v2(A: np.ndarray, B: np.ndarray, C: np.ndarray):
    Y = np.empty((16, 128, 128), dtype="float32")
    for n in range(16):
        for i in range(128):
            for j in range(128):
                for k in range(128):
                    if k == 0:
                        Y[n, i, j] = 0
                    Y[n, i, j] = Y[n, i, j] + A[n, i, k] * B[n, k, j]
    for n in range(16):
        for i in range(128):
            for j in range(128):
                C[n, i, j] = max(Y[n, i, j], 0)

In [53]:
a = np.random.randn(16,128,128)
a = a.astype("float32")
b = np.random.randn(16,128,128)
b = b.astype("float32")
c = np.empty((16,128,128))

In [54]:
lnumpy_mm_relu_v2(a,b,c)

In [56]:
# print(a.dtype)
rt_lib = tvm.build(MyBmmRelu, target="llvm")
a_tvm = tvm.nd.array(a)
b_tvm = tvm.nd.array(b)
c_tvm = tvm.nd.array(np.empty((16, 128, 128), dtype=np.float32))
rt_lib["bmm_relu"](a_tvm, b_tvm, c_tvm)
# print(conv_tvm.numpy())
np.testing.assert_allclose(c_tvm.numpy(), c, rtol=1e-5)

In [59]:
before_rt_lib = tvm.build(MyBmmRelu, target="llvm")
after_rt_lib = tvm.build(sch.mod, target="llvm")
a_tvm = tvm.nd.array(np.random.rand(16, 128, 128).astype("float32"))
b_tvm = tvm.nd.array(np.random.rand(16, 128, 128).astype("float32"))
c_tvm = tvm.nd.array(np.random.rand(16, 128, 128).astype("float32"))
after_rt_lib["bmm_relu"](a_tvm, b_tvm, c_tvm)
before_timer = before_rt_lib.time_evaluator("bmm_relu", tvm.cpu(), repeat = 100)
print("Before transformation:")
print(before_timer(a_tvm, b_tvm, c_tvm))

f_timer = after_rt_lib.time_evaluator("bmm_relu", tvm.cpu(), repeat = 100)
print("After transformation:")
print(f_timer(a_tvm, b_tvm, c_tvm))

Before transformation:
Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
  33.3152      33.0825      43.2520      32.9315       1.3656   
               
After transformation:
Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
   0.9198       0.8575       1.7892       0.8516       0.1759   
               
